In [36]:
# Imports (External)
import numpy as np
import pandas as pd
import datetime
from datetime import datetime

from monthdelta import monthdelta
import xlrd
import xlsxwriter
from collections import OrderedDict

import sys
sys.path.append('../')  

# Visualization/plotting imports
import matplotlib as mpl
import matplotlib.pyplot as plt

# Machine learning imports
# import sklearn
# import tensorflow as tf
# import keras
import pywt
from pywt import wavedec, waverec
from scipy import signal
from statsmodels.robust import mad

# Internal Imports
from wsae_lstm.utils import dictmap_load

In [37]:
dict_dataframes_index = dictmap_load(path = "../data/processed/clean_data_index.xlsx")   
dict_dataframes_future = dictmap_load(path = "../data/processed/clean_data_future.xlsx")   

In [38]:
dict_dataframes_index.keys()

odict_keys(['csi300 index data', 'nifty 50 index data', 'hangseng index data', 'nikkei 225 index data', 's&p500 index data', 'djia index data'])

In [39]:
dict_dataframes_index['csi300 index data'].head()

,date,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
0,2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
1,2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08
2,2008-07-03,2654.48,2807.68,2617.26,2760.61,456603.05,72.73,2.5762,-206.941406,-112.113057,190.424,3109.044731,2978.942155,2820.8364,-208.925,2753.2792,-82.064,-2.886857,-0.031251,-5.855737e+07
3,2008-07-04,2751.21,2783.85,2716.02,2741.85,379050.10,72.71,2.5632,-200.759162,-81.997539,67.825,3073.107115,2956.362140,2810.0548,-239.055,2738.4454,-249.421,-8.338289,-0.017237,-7.467503e+07
4,2008-07-07,2747.61,2890.99,2747.61,2882.76,527320.24,72.71,2.5679,-182.386907,21.707767,149.133,3046.256923,2949.352699,2819.3370,66.742,2756.6342,109.687,3.955427,0.010701,-3.531151e+07


In [40]:
df = dict_dataframes_index['csi300 index data']
df = df.set_index(df['date'])

In [41]:
df_train = df[(df['date'] >= '2008-7-01') & (df['date'] <= '2010-07-01')]

In [42]:
df_train.head(1)

,date,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,,
2008-07-01,2008-07-01,2799.2,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.2,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.77,-9.424605,-0.097927,-8.726278e+07


In [43]:
df_train.tail(1)

,date,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,,
2010-07-01,2010-07-01,2557.92,2574.1,2524.57,2526.07,328576.46,84.55,2.2492,-68.405561,-199.061153,49.525,2742.562654,2713.585325,2691.0766,-232.424,2626.8448,-223.952,-8.143641,-0.063088,-9.298846e+07


In [44]:
# dict_dataframes_index['csi300 index data'].drop(['date'],axis=1,inplace=True)
# dict_dataframes_index['csi300 index data'].values

In [53]:
df_val = df[(df['date'] >= '2010-7-01') & (df['date'] <= '2010-10-01')]

In [54]:
df_val

,date,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,,
2010-07-01,2010-07-01,2557.92,2574.10,2524.57,2526.07,328576.46,84.55,2.2492,-68.405561,-199.061153,49.525,2742.562654,2713.585325,2691.0766,-232.424,2626.8448,-223.952,-8.143641,-0.063088,-9.298846e+07
2010-07-02,2010-07-02,2523.04,2542.12,2462.20,2534.11,449141.00,84.37,2.1942,-74.447502,-160.377588,79.920,2729.510192,2696.491961,2674.8705,-223.393,2586.4084,-224.761,-8.146862,-0.088669,-8.628590e+07
2010-07-05,2010-07-05,2507.35,2529.43,2476.49,2512.65,321785.74,84.56,2.1650,-80.044241,-126.624360,57.615,2717.922038,2678.983298,2648.0693,-223.636,2545.5832,-230.082,-8.388786,-0.097377,-1.121958e+08
2010-07-06,2010-07-06,2504.87,2563.41,2502.93,2562.90,397147.62,84.05,2.1567,-79.508340,-82.713045,60.480,2708.266577,2667.927936,2625.9875,-153.875,2539.7598,-133.264,-4.942722,-0.070906,-4.282908e+07
2010-07-07,2010-07-07,2561.11,2580.61,2548.56,2580.48,357209.09,83.96,2.1291,-76.780401,-55.102969,32.049,2697.516500,2659.599275,2608.1857,-11.542,2543.2412,-200.186,-7.199218,-0.064196,1.266042e+07
2010-07-08,2010-07-08,2591.51,2602.23,2564.64,2575.92,340569.66,83.86,1.9842,-74.131579,-41.411486,37.592,2686.963385,2651.629916,2590.0280,12.851,2553.2112,-207.799,-7.464795,-0.058607,5.496039e+06
2010-07-09,2010-07-09,2578.56,2662.16,2557.26,2647.10,584930.45,83.98,1.7206,-65.533070,-7.052003,104.904,2682.111077,2651.198876,2581.1097,121.033,2575.8110,-111.391,-4.038108,-0.025645,2.441766e+07
2010-07-12,2010-07-12,2647.42,2689.81,2636.65,2676.22,584624.91,84.25,1.6916,-55.727047,37.734720,53.161,2679.497846,2653.581650,2577.0538,142.113,2608.5244,-81.280,-2.947600,0.031462,8.131556e+07
2010-07-13,2010-07-13,2649.66,2649.66,2618.34,2634.59,432285.06,83.52,1.6479,-50.729713,19.231024,57.878,2674.769615,2651.773207,2581.3112,121.942,2622.8626,-101.694,-3.716496,0.027148,3.920433e+07


In [48]:
# validating
#df_train['date'][0]+monthdelta(3)

Timestamp('2008-10-01 00:00:00')

In [52]:
# testing
#df_train['date'][-1]+monthdelta(6)

Timestamp('2011-01-01 00:00:00')

In [ ]:
# dict_dataframes_index['csi300 index data'].drop(['date'],axis=1,inplace=True)
# dict_dataframes_index['csi300 index data'].values

In [ ]:
# from sklearn.model_selection import TimeSeriesSplit
# X = dict_dataframes_index['csi300 index data'].values
# tscv = TimeSeriesSplit(n_splits=6)
# for train_index, test_index in tscv.split(X):
#     train = X[train_index]
#     test = X[test_index]
#     print('Observations: %d' % (len(train) + len(test)))
#     print('Training Observations: %d' % (len(train)))
#     print('Testing Observations: %d' % (len(test)))